In [ ]:
from cuml.model_selection import train_test_split
from cuml.ensemble import VotingRegressor
from cuml.ensemble import GradientBoostingRegressor
from cuml.svm import SVR
from cuml.metrics import mean_absolute_error
from cuml.model_selection import cross_val_score
import mlflow.sklearn
import pandas as pd
import mlflow
import logging
import os
from ray import tune
from ray.tune.integration.mlflow import mlflow_mixin
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.optuna import OptunaSearch

In [ ]:
wine = pd.read_csv('winequality-red.csv')

X = wine.drop('quality', axis=1)
y = wine.quality
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

logging.basicConfig(level=logging.INFO, format="%(asctime)-15s %(message)s")
logger = logging.getLogger()

In [ ]:
# define objective function
@mlflow_mixin
def finetuning_model(config):
    mlflow.sklearn.autolog()
    gbm = GradientBoostingRegressor(config['n_estimators'], config['max_depth'], config['learning_rate'])
    svr = SVR(config['C'])
    blender = VotingRegressor(random_state = 0, estimators = [('gbm', gbm), ('svm', svr)], n_jobs = -2)
    mae = cross_val_score(blender, X_train, y_train, cv = 5, scoring = "mae")

In [ ]:
def run_hpo_model(num_samples = 10, tracking_uri = None, experiment_name = 'ray_tune_optimization'):

    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(experiment_name)
    
    # define search algo and scheduler
    searcher = OptunaSearch()
    searcher = ConcurrencyLimiter(searcher, max_concurrent=4)
    scheduler = AsyncHyperBandScheduler()

    config = {
        "n_estimators": tune.choice("n_estimators", [500, 600, 700]),
        "max_depth": tune.uniform("max_depth", 2, 15),
        "learning_rate": tune.choice("learning_rate", [0.1, 0.01]),
        "C": tune.uniform("C", 5, 15)
        "mlflow": {
                "experiment_name": experiment_name,
                "tracking_uri": mlflow.get_tracking_uri()},
    }
    
    trainable = tune.with_parameters(finetuning_model)
    
    analysis = tune.run(
    trainable,
    resources_per_trial={"cpu": 1,},
    metric="mean_absolute_error",
    mode="min",
    config=config,
    num_samples=num_samples,
    search_alg=searcher,
    scheduler=scheduler,
    name="ray_tuning_model")
    
    logger.info("Best hyperparameters found were: %s", analysis.best_config)

def task():
    run_hpo_dl_model(num_samples=10,
                     tracking_uri="http://localhost",
                     experiment_name="hpo-tuning-chapter06")


if __name__ == '__main__':
    task()